# Scraping
#### Se realiza la carga de datos desde el sitio argenprop.com 

### Intalación de las librerías necesarias

In [1]:
pip install tqdm

    100% |████████████████████████████████| 81kB 1.2MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

    100% |████████████████████████████████| 102kB 1.2MB/s a 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

    100% |████████████████████████████████| 5.5MB 272kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Importamos las librerías y creamos el dataset de Pandas

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.auto import tqdm
import os

df=pd.DataFrame(columns=['Precio','Ubicacion','Barrio','Metros','Dorm','Baños','Años'])

### Obtenemos el número total de páginas de publicaciones

In [5]:
url = "https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-2"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9     (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
r = requests.get(url,headers=headers)
soup = BeautifulSoup(r.text, 'lxml')
paginas = soup.find_all('li', class_="pagination__page")
paginas = int(paginas[-2].text)
print("Total de páginas:",paginas)

Total de páginas: 854


### Recolectamos la información de cada página, la parseamos y la incuimos en el dataset

In [6]:

for j in tqdm(range(2,paginas)): # PARA MENOR TIEMPO reemplazar paginas por otro valor así no se trae la totalidad de páginas 
    url = "https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-"+str(j)
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9     (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')

    precio = soup.find_all('p',class_="card__price")
    caract = soup.find_all('ul',class_="card__main-features")
    direccion = soup.find_all('p',class_="card__title--primary")

    try:
     ind = df.index[-1]+1
    except: 
        ind = 0

    for i in range(len(precio)):

        precios = precio[i].text.replace("USD ","").split()
        caracts = caract[i].text.split()
        direcciones = direccion[i].text.replace("Departamento en Venta en ","").split(", ")
        
        df.loc[ind+i,'Ubicacion'] = direcciones[0]
        df.loc[ind+i,'Barrio'] = direcciones[1]
        
        if precios[0] == 'Consultar' :
            df.loc[ind+i,'Precio'] = 999
        else:
            df.loc[ind+i,'Precio'] = int(precios[0].replace('.',''))

        if 'm²' in caracts:
            df.loc[ind+i,'Metros'] = float(caracts[caracts.index('m²')-1].replace(',','.'))

        if 'baño' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baño')-1])

        if 'baños' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baños')-1])

        if 'Monoam.' in caracts:
            df.loc[ind+i,'Dorm'] = 0
    
        if 'dorm.' in caracts:
            df.loc[ind+i,'Dorm'] = int(caracts[caracts.index('dorm.')-1])

        if 'años' in caracts:
            df.loc[ind+i,'Años'] = int(caracts[caracts.index('años')-1])

        if 'Estrenar' in caracts:
            df.loc[ind+i,'Años'] = 0
        

  0%|          | 0/852 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: '$'

In [8]:
import findspark

findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [10]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("pyspark-postgres")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

### Convertimos los valores faltantes en '999' y luego transformamos el Dataframe de Pandas en Spark

In [11]:
df=df.fillna(999)
df = spark.createDataFrame(df)

### Guardamos en Postgres

In [12]:
df \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [13]:
df_pos = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .load()

### Corroboramos que coincidan la cantidad de filas y columnas y mostramos las primeras 100 filas del Dataframe importado de postgres

In [14]:
print((df.count(), len(df.columns)))
print((df_pos.count(), len(df_pos.columns)))

df_pos.show(100)

(871, 7)
(871, 7)
+-------+------------------+---------------+------+----+-----+-----+
| Precio|         Ubicacion|         Barrio|Metros|Dorm|Baños| Años|
+-------+------------------+---------------+------+----+-----+-----+
| 399900|      Villa Devoto|Capital Federal| 115.0|   3|    1|   41|
| 211000|      Villa Crespo|Capital Federal|  98.0|   4|    2|   47|
| 137000|  Villa Santa Rita|Capital Federal|  50.0|   1|    1|   10|
|  37600|            Centro|Capital Federal|  19.0| 999|    1|  999|
| 390000|             Nuñez|Capital Federal| 145.0|   3|    3|   35|
| 100000|     Palermo Viejo|        Palermo|  35.0|   0|    1|   20|
| 980000|      Barrio Norte|Capital Federal| 135.0|   2|    3|  999|
| 315000|     Villa Urquiza|Capital Federal|  74.0|   2|    2|    7|
| 550000|           Palermo|Capital Federal| 110.0|   2|    3|    0|
| 140000|  Parque Chacabuco|Capital Federal|  82.0|   2|    2|   35|
| 120000|  Villa del Parque|Capital Federal|  45.0|   1|    1|   14|
| 115000|       

In [181]:
spark.stop()